# Quake 3 commands

For server and client, for running, configuring, downloading, installing, cloudify, scriptifying, copying, etc.




## special use cases


### On Mac it is necessary to quaratine apps

That have been downloaded and moved to /Applications when they need to access their own database inside the .app directory:


In [ ]:
xattr -d com.apple.quarantine /Applications/ioquake3/ioquake3.app

open /Library/Developer/CommandLineTools/Packages/macOS_SDK_headers_for_macOS_10.14.pkg

sudo codesign --force --deep --sign - path-to-app.app


### OpenArena on Mac/Linux

Requires a chmod because the build process should do this instead:


In [ ]:
chmod +x /Applications/openarena-0.8.8/OpenArena.app/Contents/MacOS/openarena.ub
chmod +x "/Applications/openarena-0.8.8/OpenArena 0.8.8 r28.app/Contents/MacOS/openarena.ub"



## server commands

Starting, managing servers, etc.


### starting the server


In [1]:
open -n ./ioquake3-1.36.app --args +set fs_game baseq3 +set dedicated 1 +exec ./server.cfg


open ./build/release-darwin-x86_64/ioquake3.app/ --args +set dedicated 1 +set fs_basepath ~/.q3a/unpacked +set fs_basegame baseq3-combined-converted +set fs_homepath ~/.q3a/ +map q3dm0

./build/release-darwin-x86_64/ioq3ded.x86_64 --args +set ttycon 0 +set fs_basepath /Users/briancullinan/planet_quake_data +set fs_basegame baseq3-combined-converted-repacked +set fs_game baseq3-combined-converted-repacked-menu +set com_gamename Quake3Arena +set sv_pure 1 +map q3tourney5


The file /Users/briancullinan/jupyter_ops/Games/ioquake3-1.36.app does not exist.


: 1

### creating a VNC quake 3 client

Try streaming the game over VNC, is it awful? Kind of, and inputs don't work as expected.



In [ ]:

# working vnc key
# VND7Z-M22AB-MQD7R-GAAKF-2B3EA


docker run -it --shm-size=2g --name quake3 -d -p 5901:5900 quake3 bash


#### client script to start quake as a player


In [ ]:
#!/bin/bash

export DISPLAY=:4000

/usr/bin/Xvfb $DISPLAY -screen 0 1000x600x24 -dpi 70 -ac +extension RANDR & echo $! > /tmp/xvfb.pid
/usr/bin/xinit -display $DISPLAY & echo $! > /tmp/x11.pid

/usr/lib/ioquake3/ioquake3 --args +set fs_game missionpack +exec ./client.cfg & echo $! > /tmp/xterm.pid

sleep 5
/usr/bin/x11vnc -scale 1000x600 -noxdamage -forever -shared -rfbport 5900 -display $DISPLAY -noxrecord & echo $! > /tmp/vnc.pid

int_handler()
{
    kill $(cat /tmp/xterm.pid) 2> /dev/null
    kill $(cat /tmp/vnc.pid)
    kill $(cat /tmp/x11.pid)
    kill $(cat /tmp/xvfb.pid)
}

trap 'int_handler' INT

wait $(cat /tmp/xterm.pid)
int_handler



#### systemd configurations for quake 3 services



In [ ]:
# quake3-proxy
[Unit]
Description=Quake proxy service.

[Service]
Type=simple
ExecStart=/usr/bin/node /home/megamindbrian/proxy.js
Restart=on-failure

[Install]
WantedBy=multi-user.target


#quake3-server
[Unit]
Description=Quake server.

[Service]
Type=simple
ExecStart=/home/megamindbrian/ioq3-master/build/release-linux-x86_64/ioq3ded.x86_64 +set fs_homepath /home/megamindbrian/.q3a +set com_hunkMegs 312 +exec server.cfg
Restart=on-failure

[Install]
WantedBy=multi-user.target



### Done: play quake 3

https://quake.games/?set%20name%20player_name

clone of https://openarena.live/

or http://www.quakejs.com/play?set%20fs_game%20cpma&set%20mode_start%20FFA&set%20g_teamAutoJoin%201&map%20cpm1a




## client commands

Done: drop in to quake 3 browser with server backend. configured. shared. hosted. offer to invite friends with a link.



### starting the client



In [ ]:
open -n ./ioquake3-1.36.app --args +set fs_game baseq3 +exec ./client.cfg

open -n /Users/briancullinan/planet_quake/build/release-darwin-x86_64/ioquake3.app --args +set fs_basepath /Applications/ioquake3 +set fs_homepath /Users/briancullinan/.q3a +set fs_restrict 0 +set sv_pure 0 +set sv_cheats 1 +set cheats 1 +set developer 1 +set logfile 1 +set fs_game baseq2vm +map base2-a

open -n /Users/briancullinan/planet_quake/build/release-darwin-x86_64/ioquake3.app --args +set fs_basepath /Applications/ioquake3 +set fs_homepath /Users/briancullinan/.q3a +set fs_restrict 0 +set sv_pure 0 +set sv_cheats 1 +set cheats 1 +set developer 1 +set logfile 1 +set fs_game baseq2vm +set r_showtris 0 +devmap base2-a

open -n ./build/release-darwin-x86_64/ioquake3.app --args +set fs_basepath /Users/briancullinan/.q3a/unpacked +set fs_homepath /Users/briancullinan/.q3a/unpacked +set fs_restrict 0 +set sv_pure 0 +set sv_cheats 1 +set cheats 1 +set developer 1 +set logfile 1 +set fs_game baseq3-combined-converted +devmap "Full stadium 2"


open -n /Users/briancullinan/planet_quake/build/release-darwin-x86_64/ioquake3.app --args +set fs_basepath /Users/briancullinan/.q3a/unpacked +set fs_homepath /Users/briancullinan/.q3a +set fs_restrict 0 +set sv_pure 0 +set sv_cheats 1 +set cheats 1 +set developer 1 +set logfile 1 +set fs_game baseq3-combined-converted


./build/debug-darwin-x86_64/quake3e +set fs_basepath /Applications/ioquake3 +set fs_game baseq3 +set sv_pure 0 +set net_enable 1 +set recentPassword "recentPassword" +set timescale 8 +set developer 1 +set dedicated 0 +bind leftarrow rewind +bind rightarrow forward +demo baseq3_1_map_2_matches



## mapping commands

Commands for building maps, managing assets, converting designs, etc.



### build quake 3 maps (q3map2)

In [ ]:
q3map2.exe -convert -light -format map -fs_basepath Z:\.q3a -game baseq3 -v Z:\.q3a\baseq3\acid3dm5.pk3dir\maps\acid3dm5.bsp


q3map2.exe -convert -light -format map -fs_basepath Z:\.quake2 -game baseq2 -v Z:\.quake2\baseq2\pak0.pak\maps\base1.bsp



./q3map2 -meta -v -fs_basepath /Applications/ioquake3/unpacked -game baseq3 /Applications/ioquake3/unpacked/acid3dm5.pk3dir/maps/acid3dm5_converted.map
./q3map2 -vis -v -fs_basepath /Applications/ioquake3/unpacked -game baseq3 /Applications/ioquake3/unpacked/acid3dm5.pk3dir/maps/acid3dm5_converted.map

./q3map2 -bsp -meta -patchmeta -v -fs_basepath /Applications/ioquake3/unpacked -game baseq3 /Applications/ioquake3/unpacked/acid3dm5.pk3dir/maps/acid3dm5_converted.map

./q3map2 -convert ase -v -fs_basepath /Applications/ioquake3/unpacked -game baseq3 /Applications/ioquake3/unpacked/acid3dm5.pk3dir/maps/acid3dm5_converted.map     

./q3map2 -light -fast -patchshadows -samples 3 -bounce 8 -gamma 2 -compensate 4 -dirty -v -fs_basepath /Applications/ioquake3/unpacked -game baseq3 /Applications/ioquake3/unpacked/acid3dm5.pk3dir/maps/acid3dm5_converted.map

/Applications/GtkRadiant.app/Contents/Resources/install/q3map2 -game quake3 -fs_basepath /Applications/ioquake3/ -fs_game baseq2 -meta ~/planet_quake_data/quake2-baseq2/pak0.pk3dir/maps/base1.map

/Applications/GtkRadiant.app/Contents/Resources/install/q3map2 -game quake3 -fs_basepath /Applications/ioquake3/ -fs_game baseq3 -light _keeplights  -fast ~/planet_quake_data/quake2-baseq2/pak0.pk3dir/maps/base1.map

/Applications/GtkRadiant.app/Contents/Resources/install/q3map2 -game quake3 -fs_basepath /Applications/ioquake3/ -fs_game baseq3 -vis _keeplights ~/planet_quake_data/quake2-baseq2/pak0.pk3dir/maps/base1.map

# normalize a bsp vectors all at once
/Applications/NetRadiant.app/Contents/MacOS/install/q3map2.bin -game quake3 -fs_basepath /Applications/ioquake3/ -fs_game baseq2 -convert -format map -readmap ~/planet_quake_data/quake2-baseq2/pak0.pk3dir/maps/base2.map


/Applications/GtkRadiant.app/Contents/Resources/install/q3map2 -game quake3 -fs_basepath /Applications/ioquake3/ -fs_game baseq2 -meta _keeplights ~/planet_quake_data/quake2-baseq2/pak0.pk3dir/maps/base2-a.map



### convert a bsp to map


TODO: figure out where lights come from and add them to converted map since they are removed for quake 3 maps (unecessary because of how lightmaps work, but makes decompiling worse)



#### convert a quake 2 bsp to map


In [ ]:


Z:\Downloads\netradiant-custom-win64\netradiant-custom-20190705\mbspc.exe -bsp2map Z:\.quake2\baseq2\pak0.pak\maps\base1.bsp

for /r %%v in (*.bsp) do (
    cd "%%~dpv"
    "Z:\planet_quake_data\tools\mbspc.exe" -bsp2map "%%~nv.bsp")

for /r %%v in (*.md2) do "Z:\planet_quake_data\tools\qwalk\modelconv.exe" -i "%%v" "%%~dpv%%~nv.md3"


/Applications/GtkRadiant.app/Contents/Resources/install/q3map2 -game quake3 -fs_basepath /Applications/ioquake3/ -fs_game baseq3 -convert -format map ~/planet_quake_data/quake3-baseq3/pak0.pk3dir/maps/q3tourney6.bsp

for i in $(find "$(pwd -P)" -iname '*.bsp'); \
do if [ ! -f "${i%.*}.map" ]; \
then pushd $(dirname "$i"); \
/Users/briancullinan/netradiant-custom/tools/mbspc/build/macosx/mbspc -bsp2map $(basename "$i"); \
popd; \
fi; \
done;

# normalize brushes verteces using q3map2
for i in $(find "$(pwd -P)" -iname '*.bsp'); \
do if [ ! -f "${i%.*}_converted.map" ]; \
then pushd $(dirname "$i"); \
/Applications/NetRadiant.app/Contents/MacOS/install/q3map2.bin -game quake3 -fs_basepath /Applications/ioquake3/ -fs_game baseq2 -convert -format map -readmap $(basename "${i%.*}.map")
popd; \
fi; \
done;


# run the entire process for a single map
i="/Users/briancullinan/planet_quake_data/quake2-baseq2/pak0.pk3dir/maps/base2.map"; \
pushd $(dirname "$i"); \
/Users/briancullinan/planet_quake_data/tools/mbspc/build/macosx/mbspc -bsp2map $(basename "${i%.*}.bsp"); \
/Applications/NetRadiant.app/Contents/MacOS/install/q3map2.bin -game quake3 -fs_basepath /Applications/ioquake3/ -fs_game baseq2 -convert -format map -readmap $(basename "${i%.*}.map");

# convert acting on nicely formated verteces
pushd /Users/briancullinan/jupyter_ops/; \
npm run import "convert quake 2 map" -- "['${i%.*}_converted.map']"; \
npm run import "split map hints" -- "['${i%.*}_converted.map']";

# compile hinted maps
/Applications/GtkRadiant.app/Contents/Resources/install/q3map2 -game quake3 -fs_basepath /Applications/ioquake3/ -fs_game baseq2 -meta _keeplights "${i%.*}-a.map"; \
/Applications/GtkRadiant.app/Contents/Resources/install/q3map2 -game quake3 -fs_basepath /Applications/ioquake3/ -fs_game baseq2 -meta _keeplights "${i%.*}-b.map"; \
/Applications/GtkRadiant.app/Contents/Resources/install/q3map2 -game quake3 -fs_basepath /Applications/ioquake3/ -fs_game baseq2 -meta _keeplights "${i%.*}-c.map";

/Applications/NetRadiant.app/Contents/MacOS/install/q3map2.bin -game quake3 -fs_basepath /Applications/ioquake3/ -fs_game baseq2 -meta _keeplights "${i%.*}_converted.map"

popd
popd




#### convert textures to TGA


In [ ]:

for /r %%v in (*.wal) do "Z:\planet_quake_data\tools\Wal2TGA\wal2tga copy.exe" "%%v" "%%~dpv%%~nv.tga"

# TODO: bash uses glob, windows can use "or" keyword
#windows
for /r %%v in (*.pcx or *.jpg or *.jpeg) do convert -colorspace RGB "%%v" "%%~dpv%%~nv.tga" ; done

#bash
for i in $(find . -iname '*.pcx' -o -iname '*.jpg' -o -iname '*.jpeg' -o -iname '*.tga'); \
do if [ ! -f "${i%.*}.tga" ]; \
then convert -colorspace RGB "$i" "${i%.*}.tga"; \
fi; \
done;


convert -colorspace RGB pak0.pk3dir/models/deadbods/dude/dead1.pcx pak0.pk3dir/models/deadbods/dude/dead1.tga


#### convert wav files to opus for use with web



In [ ]:

for /r %%v in (*.wav) do opusenc "%%v" "%%~dpv%%~nv.opus"

#TODO: WOOOO! Bash pattern, convert this with transpiler to match linux loops with same filename extension operation
#bash
for i in $(find . -iname '*.wav'); do opusenc "$i" "${i%.wav}".opus; done

for i in $(find . -iname '*.wav'); do echo "$i"; done


### TODO: collect these doom maps


In [ ]:

-q3twodeaths (dm1)
-q3claustro (dm2)
-q3badplace (dm4)
-q3cistern (dm5)
-q3darkzone (dm6)
-q3death32c (original map by (i dunno))
-q3ultrav (original map by escher)
-q3abandoned (dm3)
-q3chthon (e1m7)
-q3start (introduction)
-q3debello (original map by escher)



### TODO: get map2q3 ported to javascript 

Because of "brush line limit" error, lame. Can't recompile either.


In [ ]:



Z:\Downloads\map2q3\bin\win32\map2q3.exe -s Z:\.quake2\baseq2\pak0.pak\maps\base1.bsp -q2



## building commands



### building game files


### building the engine

In [ ]:
cmake -G "Unix Makefiles" -H. -Bbuild -DCMAKE_BUILD_TYPE=Release -DCMAKE_C_COMPILER=/usr/local/bin/gcc-9 -DCMAKE_CXX_COMPILER=/usr/local/bin/g++-9

./code/xquakejs/lib/emsdk/upstream/emscripten/emcc -O1 -s STANDALONE_WASM -s EXPORTED_FUNCTIONS="['_MSG_initHuffman', '_Huff_Decompress', '_Huff_Compress']" -s EXPORT_NAME="huffman" ./code/xquakejs/lib/huffman.c -o ./code/xquakejs/lib/huffman.js

./code/xquakejs/lib/emsdk/upstream/emscripten/emcc -O1 -s SAFE_HEAP=1 -s WASM=0 -s EXTRA_EXPORTED_RUNTIME_METHODS="['getValue', 'allocate']" -s EXPORTED_FUNCTIONS="['_MSG_initHuffman', '_Huff_Decompress', '_Huff_Compress', '_HuffmanGetSymbol']" -s EXPORT_NAME="huffman" ./code/xquakejs/lib/huffman.c -o ./code/xquakejs/lib/huffman.js

./dpmaster -L --log-file /Users/briancullinan/Downloads/dpmaster-2.2/dpmaster.log -v -g Quake3Arena options=send-empty-servers



### misc commands



#### extract linux quake3 image


In [ ]:

tail -n +165 linuxq3ademo.gz.sh | gzip -cd | tar xf -



### build script

In [ ]:
#bash
for i in $(find . -iname '*.bsp' -o -iname '*.map'); do
if [[ ! "$i" =~ concepts/ ]]; then
if [[ ! "$i" =~ autosave/ ]]; then
if [[ ! "$i" =~ quarantine/ ]]; then

if [ ! -f "${i%.*}.map" ]; then
if [ -f "${i%.*}_converted.map" ]; then
mv "${i%.*}_converted.map"  "${i%.*}.map";
fi;
fi;


if [ ! -f "${i%.*}.map" ]; then
if [ -f "${i%.*}.bsp" ]; then
~/Quake3e/build/release-darwin-arm/quake3e_q3map2 -convert -format map -keeplights -game quake3 -fs_basepath /Users/briancullinan/planet_quake_data -fs_game quake3-baseq3 "${i%.*}.bsp";
mv "${i%.*}_converted.map"  "${i%.*}.map";
fi;
fi;


# comment out this line and third fi; to replace existing bsps?
if [ ! -f "${i%.*}.bsp" ]; then
if [ -f "${i%.*}.map" ]; then
if [ $(wc -c <"${i%.*}.map") -le 100000 ]; then
~/Quake3e/build/release-darwin-arm/quake3e_q3map2 -meta -keeplights -game quake3 -fs_basepath /Users/briancullinan/planet_quake_data -fs_game quake3-baseq3 "${i%.*}.map";
~/Quake3e/build/release-darwin-arm/quake3e_q3map2 -light -fast -game quake3 -fs_basepath /Users/briancullinan/planet_quake_data -fs_game quake3-baseq3 "${i%.*}.bsp";
fi;
fi;
fi;


# if [ -f "${i%.*}.bsp" ]; then
# if [ $(wc -c <"${i%.*}.bsp") -le 900000 ]; then
# /Applications/Blender.app/Contents/MacOS/Blender --python ../model-convert.py -- "${i%.*}.bsp";
# fi;
# fi;

# if it still doesn't exist 

# if [ ! -f "${i%.*}.md3" ]; then
# if [ ! -f "${i%.*}-half.md3" ]; then

# if [ -f "${i%.*}.map" ]; then
# if [ $(wc -c <"${i%.*}.map") -le 900000 ]; then
# /Applications/Blender.app/Contents/MacOS/Blender --python ../model-convert.py -- "${i%.*}.map";
# fi;
# fi;

# fi;
# fi;

fi;
fi;
fi;
done;

